In [22]:
import pandas as pd
import jieba
from collections import defaultdict
import warnings
import os
from copy import deepcopy
import re
from torch.utils.data import TensorDataset
import torch
from collections import Counter
import numpy as np
from transformers import BertTokenizer,BertConfig
from pandas.core.frame import DataFrame

import numpy as np
from numpy.random import randn

In [23]:
bert_path = 'hfl/chinese-bert-wwm-ext'
tokenizer = BertTokenizer.from_pretrained(bert_path)

In [24]:
MAX_LEN = 512
PARTITION = 10

In [41]:
#jieba.load_userdict('./data/diag_dict.txt') 
#data_name = 'train'
#data_name = 'dev'
data_name = 'test'

data_path = './data/hospital_{}_df_{}.pkl'.format(data_name, PARTITION)
sample_df = pd.read_pickle(data_path)

In [42]:
sample_df['no_sym_describe'].fillna(" ",inplace=True)

In [43]:
# 将各个特征分词，加上special token，并转为fillnat_ids

sample_df['sym_input_ids'] = sample_df['sym_describe'].apply(lambda x : tokenizer.encode(x,add_special_tokens = True,max_length = MAX_LEN,
                                                                                             truncation = True))
sample_df['no_sym_input_ids'] = sample_df['no_sym_describe'].apply(lambda x : tokenizer.encode(x,add_special_tokens = True,max_length = MAX_LEN,
                                                                                             truncation = True))

In [44]:
# 获得各个特征的attention_mask
sample_df['sym_attention_mask'] = sample_df['sym_input_ids'].apply(lambda x : len(x) * [1] + (MAX_LEN - len(x)) * [0] )
sample_df['no_sym_attention_mask'] = sample_df['no_sym_input_ids'].apply(lambda x : len(x) * [1] + (MAX_LEN - len(x)) * [0] )

# 将xxx_input_ids填充
sample_df['sym_input_ids'] = sample_df['sym_input_ids'].apply(lambda x : x + (MAX_LEN - len(x)) * [0] )
sample_df['no_sym_input_ids'] = sample_df['no_sym_input_ids'].apply(lambda x : x + (MAX_LEN - len(x)) * [0] )

In [45]:
# 定位sep的索引，从而获得token_type_ids,0留给cls
def get_token_type_ids(sym_input_ids):
    token_type_ids = [0]
    loc_list = [0] + list(np.where(np.array(sym_input_ids) == 102)[0])
    for i in range(len(loc_list) - 1):
        token_type_ids += [i + 1] * (loc_list[i + 1] - loc_list[i])
    
    token_type_ids += (MAX_LEN - len(token_type_ids)) * [-1]  
    return token_type_ids

# 获得各部分token_type从而区分organ

sample_df['sym_token_type_ids'] = sample_df['sym_input_ids'].apply(lambda x : get_token_type_ids(x))
sample_df['no_sym_token_type_ids'] = sample_df['no_sym_input_ids'].apply(lambda x : get_token_type_ids(x))

In [46]:
# 随机打乱样本顺序
sample_df = sample_df.sample(n = len(sample_df),random_state = 42).reset_index(drop=True)

In [47]:
# 构建tensordataset数据集并保存
sym_input_ids = torch.tensor(list(sample_df['sym_input_ids']), dtype=torch.long)
no_sym_input_ids = torch.tensor(list(sample_df['no_sym_input_ids']), dtype=torch.long)

sym_attention_mask = torch.tensor(list(sample_df['sym_attention_mask']), dtype=torch.long)
no_sym_attention_mask = torch.tensor(list(sample_df['no_sym_attention_mask']), dtype=torch.long)


sym_token_type_ids = torch.tensor(list(sample_df['sym_token_type_ids']), dtype=torch.long)
no_sym_token_type_ids = torch.tensor(list(sample_df['no_sym_token_type_ids']), dtype=torch.long)

all_labels = torch.tensor(sample_df['label'], dtype=torch.long)

all_dataset = TensorDataset(sym_input_ids, no_sym_input_ids, sym_attention_mask, 
                            no_sym_attention_mask, sym_token_type_ids, 
                            no_sym_token_type_ids, all_labels)

/tmp/ipykernel_254880/2035788198.py:12: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  all_labels = torch.tensor(sample_df['label'], dtype=torch.long)


In [48]:
torch.save(all_dataset,'./data2/hospital_{}_dataset_{}_{}'.format(data_name,MAX_LEN,PARTITION))